## Column Transformer

- previously we were applying different encoder to different type of columns but instend of all those we can just use Column Transform 

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [7]:
df.shape

(100, 6)

In [4]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [5]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [6]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [8]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split( df.drop(columns= ['has_covid']),
                                                 df['has_covid'],
                                                 test_size = 0.2,
                                                 random_state = 0)
X_train.shape, X_test.shape

((80, 5), (20, 5))

## 1. Aam Zindagi --> without using Column Transformer

In [11]:
# adding simple imputer to fever col

si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])


# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [12]:
# Ordinal encoding --> cough col

oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# aslo the test data 

X_test_cough = oe.fit_transform(X_test[['cough']])


X_train_cough.shape

(80, 1)

In [13]:
# OneHot encoding --> gender, city

ohe = OneHotEncoder(drop='first', sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# aslo the test data 

X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])


X_train_gender_city.shape

(80, 4)

In [14]:
# Extract Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# aslo the test data 
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [15]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough), axis = 1)

# aslo the test data 
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough), axis = 1)

X_train_transformed.shape

(80, 7)

## Mentos zindagi

In [16]:
from sklearn.compose import ColumnTransformer

In [17]:
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild','Strong']]), ['cough'] ),
    ('tnf3', OneHotEncoder(sparse=False, drop='first'), ['gender','city'] )
], remainder='passthrough')

In [18]:
transformer.fit_transform(X_train)

array([[ 99.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  22.        ],
       [104.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  56.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  31.        ],
       [104.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  75.        ],
       [ 99.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  72.        ],
       [ 99.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  66.        ],
       [101.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  14.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  10.        ],
       [ 98.        ,   0.        ,   1.        ,   0.        ,
          1.    

In [19]:
transformer.fit_transform(X_train).shape

(80, 7)

In [20]:
transformer.transform(X_test)

array([[100.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  19.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  25.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  42.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  81.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,   5.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  27.        ],
       [103.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  69.        ],
       [ 98.        ,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  34.        ],
       [ 99.        ,   0.        ,   0.        ,   0.        ,
          0.    

In [21]:
transformer.transform(X_test).shape

(20, 7)